Scenario

You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

Instructions

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class. Here is the list of steps to be followed (building a simple model without balancing the data):

Round 1

 Import the required libraries and modules that you would need.

In [35]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

Read that data into Python and call the dataframe churnData.

In [36]:
churnData = pd.read_csv("/Users/jon/Documents/IRONHACK/DAFT_0116/module_3/LAB Imbalanced Classification Task/DATA_Customer-Churn.txt")

In [37]:
churnData.shape

(7043, 16)

Check the datatypes of all the columns in the data.You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.


In [38]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [39]:
churnData["TotalCharges"] = pd.to_numeric(churnData["TotalCharges"], errors='coerce')

Check for null values in the dataframe. Replace the null values.

In [40]:
churnData.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [41]:
churnData.fillna(churnData.median(), inplace=True)
churnData.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:

Split the data into a training set and a test set.

Scale the features either by using MinMaxScaler or a standard scaler.

In [42]:
churnData.loc[(churnData['Churn']=="No"), 'Churn'] = '0'
churnData.loc[(churnData['Churn']=="Yes"), 'Churn'] = '1'



In [67]:
X = churnData[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
y = churnData["Churn"]

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)

In [76]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# transform data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) #No need to use fit since we should normalize the training set and then use the same normalization steps on the test set


ROUND 2

(Optional) Fit a logistic Regression model on the training data.

In [77]:
log_model = LogisticRegression() 

#trans=PowerTransformer() #Optional transformation
#trans.fit(X_train_scaled)
#X_train_mod = trans.transform(X_train_scaled)
#X_test_mod  = trans.transform(X_test_scaled)
log_model.fit(X_train_scaled, y_train)
log_y_pred_test = log_model.predict(X_test_scaled) # predict test
log_y_pred_train=log_model.predict(X_train_scaled) # predict train (for sanity checks)


Fit a Knn Classifier (NOT KnnRegressor please!)model on the training data.

In [47]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=2,weights='uniform') # declare we're using knn classification model
knn_model.fit(X_train, y_train) # train model
knn_y_pred = knn_model.predict(X_test) # predict test
knn_y_pred_train=knn_model.predict(X_train) # predict train (for sanity checks)
#model.predict_proba(inputdata)

Fit a Decision Tree Classifier on the training data.

In [48]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(max_depth=3) #Comes from sklearn.tree

dt_model.fit(X_train, y_train)

y_pred_train_dt = dt_model.predict(X_train)
y_pred_test_dt = dt_model.predict(X_test)

Compare the accuracy, precision, recall for the previous models on both the train and test sets.

In [53]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt


In [83]:
performance_log = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, log_y_pred_train),
                                         precision_score(y_train, log_y_pred_train, pos_label='1'),
                                         recall_score(y_train, log_y_pred_train, pos_label='1')],
                               'Test': [accuracy_score(y_test, log_y_pred_test),
                                        precision_score(y_test, log_y_pred_test, pos_label='1'),
                                        recall_score(y_test, log_y_pred_test, pos_label='1')]})

display(performance_log)

performance_knn = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, knn_y_pred_train),
                                         precision_score(y_train, knn_y_pred_train, pos_label='1'),
                                         recall_score(y_train, knn_y_pred_train, pos_label='1')],
                               'Test': [accuracy_score(y_test, knn_y_pred),
                                        precision_score(y_test, knn_y_pred, pos_label='1'),
                                        recall_score(y_test, knn_y_pred, pos_label='1')]})

display(performance_knn)

performance_dt = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, y_pred_train_dt),
                                         precision_score(y_train, y_pred_train_dt, pos_label='1'),
                                         recall_score(y_train, y_pred_train_dt, pos_label='1')],
                               'Test': [accuracy_score(y_test, y_pred_test_dt),
                                        precision_score(y_test, y_pred_test_dt, pos_label='1'),
                                        recall_score(y_test, y_pred_test_dt, pos_label='1')]})

display(performance_dt)

,Error_metric,Train,Test
0,Accuracy,0.788782,0.801987
1,Precision,0.645975,0.692607
2,Recall,0.446680,0.470899


,Error_metric,Train,Test
0,Accuracy,0.865637,0.765791
1,Precision,0.989333,0.634831
2,Recall,0.497653,0.298942


,Error_metric,Train,Test
0,Accuracy,0.784523,0.784244
1,Precision,0.647184,0.672897
2,Recall,0.408451,0.380952
